The basic procedures include:
(1) Import packages
(2) Initialize the parameters for tracking
(3) Prepare data and models
(4) Optimize the segmentation parameters
(5) Track the following volumes

# Step 1. Import packages

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import warnings
warnings.filterwarnings('ignore')
from SiQ3D.immuneTracker import Tracker
%matplotlib inline

# Step 2. Initialize the parameters for tracking

Image parameters:\
•	volume_num: the number of volumes (i.e., time points) of the images to be tracked.\
•	xyz_size: size of a 3D frame: (height, width, depth), unit: voxels.\
•	z_xy_ratio: resolution (um/voxel) ratio between the z (depth) and x-y plane.\
Segmentation parameters:\
•	min_size: the minimal cell size; used for removing the small objects.\
•	min_distance: the minimum value of the distance used in watershed for cell segmentation.\
•	touch_ratio: the threshold value of the touching area ratio between two neighboring voxels that is used in TASCAN segmentation algorithm.\
•	noise_level: the intensity of the non-cell regions; used in local contrast normalization.\
•	threshold_method: value of 0 or 1; 0 indicates conventional threshold method, while 1 indicates OTSU threshold method. If the parameter unet_model_file is provided, the program will use the 3D U-Net, not the threshold method.\
Tracking parameters:\
•	search_radius: search radius used in the Bayesian tracking algorithm.\
File information:\
•	folder_path: the path of the folder to store data, model and the tracking results. We recommend users to create a folder under the same directory containing the jupyter notebook file, i.e., “./xxx” (“xxx” is the folder name).\
•	image_name: file name of the images to be tracked. Users should name their image sequences in the format of “xxx_Tk_Zm.tif”, where “xxx” is the prefix of the file name, and “k” and “m” are the index number of time point and z stack, respectively. For example, “nkcs_T123_Z023” indicates this is an NK cell image at time point #123 and z-stack #23. “k” and “m” are n-bit integers (e.g., “k” is a 3-bit integer if the dataset consists of hundreds of time points in total). With “k” and “m” as n-bit integers, the parameter image_name should be set in the Python format as,
 image_name=“xxx_T%0ni_Z%0ni.tif”\
•	classifier_file: file name of the pre-trained classifier.\
•	track_config_file: configuration file used by btrack.\
•	unet_model_file: file name of the pre-trained 3D U-Net model. If users do not provide this information, the program will use the threshold segmentation method, not the 3D U-Net.\


In [ ]:
tracker = Tracker(volume_num=10, xyz_size=(1024, 1024, 27), 
                  z_xy_ratio=4.6, noise_level=60, min_size=50, 
                  min_distance=20, touch_ratio=0.6, 
                  folder_path=os.path.abspath("./nkcs_tracking"), 
                  image_name="nkcs_t%03i_z%03i.tif", 
                  track_config_file = "cell_track_config.json"
                  classifier_file="nkcs_classifier_pretrained.h5", 
                  threshold_method=0, search_radius=20,
                  tascan_=1)

Multiple folders will be automatically created to store data, models and the tracking results:\
data: to store the images to be tracked.\
segmentation_results: to store the segmented images before phenotyping and tracking.\
track_results: to store the tracking results (image of labels).\
models: to store the pre-trained models.\
track_information: to store the spatial coordinates of the tracked cells.

# Step 3. Prepare data and models

Move the 2D image (z-stacks) sequences to the created folder data.\
Move the pre-trained 3D U-Net, classifier files and tracking configuration file to the created folder models.

# Step 4. Optimize the segmentation parameters

Segment cells at volume 1 and display the results.

In [ ]:
tracker.segment_vol1()
tracker.draw_segresult()

If users find the segmentation results poor, they can reset the segmentation parameters, and then re-segment volume 1 as follows.

In [ ]:
tracker.set_segmentation(min_size=80, min_distance=3, touch_ratio=0.6, touch_area=10, noise_level=45)
tracker.segment_vol1()
tracker.draw_segresult()

# Step 5. Track the following volumes

In [ ]:
tracker.track()

Save the cell coordinates and phenotyping results as csv file in the folder tracking_information.

In [ ]:
tracker.save_coordinates()